<a href="https://colab.research.google.com/github/Ashfagithub/twixia_internship_assignment_ashfa/blob/main/Twixia_Global_Internship_Assignment_Ashfa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Installing kaggle*

In [25]:
!pip install kaggle

In [26]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [19]:
!cp kaggle.json ~/.kaggle/

In [20]:
! chmod 600 ~/.kaggle/kaggle.json

In [27]:
!kaggle datasets download -d anmolkumar/house-price-prediction-challenge

  0% 0.00/2.13M [00:00<?, ?B/s]
100% 2.13M/2.13M [00:00<00:00, 145MB/s]


In [28]:
!unzip house-price-prediction-challenge.zip

Archive:  house-price-prediction-challenge.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [31]:
!ls

house-price-prediction-challenge.zip  sample_data	     test.csv
kaggle.json			      sample_submission.csv  train.csv


In [33]:
df = pd.read_csv('./train.csv')

In [34]:
df_test = pd.read_csv('./test.csv')

In [35]:
df_test.shape

(68720, 11)

In [36]:
df.shape

(29451, 12)

In [37]:
df.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [38]:
df.columns.tolist()

['POSTED_BY',
 'UNDER_CONSTRUCTION',
 'RERA',
 'BHK_NO.',
 'BHK_OR_RK',
 'SQUARE_FT',
 'READY_TO_MOVE',
 'RESALE',
 'ADDRESS',
 'LONGITUDE',
 'LATITUDE',
 'TARGET(PRICE_IN_LACS)']

In [39]:
df_no_duplicates = df.drop_duplicates()
df=df_no_duplicates
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29050 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29050 non-null  object 
 1   UNDER_CONSTRUCTION     29050 non-null  int64  
 2   RERA                   29050 non-null  int64  
 3   BHK_NO.                29050 non-null  int64  
 4   BHK_OR_RK              29050 non-null  object 
 5   SQUARE_FT              29050 non-null  float64
 6   READY_TO_MOVE          29050 non-null  int64  
 7   RESALE                 29050 non-null  int64  
 8   ADDRESS                29050 non-null  object 
 9   LONGITUDE              29050 non-null  float64
 10  LATITUDE               29050 non-null  float64
 11  TARGET(PRICE_IN_LACS)  29050 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.9+ MB


In [40]:
df.value_counts()

POSTED_BY  UNDER_CONSTRUCTION  RERA  BHK_NO.  BHK_OR_RK  SQUARE_FT    READY_TO_MOVE  RESALE  ADDRESS                    LONGITUDE  LATITUDE   TARGET(PRICE_IN_LACS)
Builder    0                   0     1        BHK        422.153370   1              0       Hoshangabad Road,Bhopal    22.750000  77.720000  10.9                     1
Owner      0                   0     1        BHK        646.678425   1              1       Amroli,Surat               21.239940  72.856520  11.0                     1
                                                         650.026001   1              1       Saravannapatti,Coimbatore  11.000000  76.966670  25.0                     1
                                                                                             Lokmanya Nagar,Palghar     22.691954  75.846950  25.0                     1
                                                                                             Katargam,Surat             21.232813  72.832454  25.0              

In [41]:
#df = df + df_test

In [42]:
df = df.drop(['ADDRESS','RERA'], axis=1)

In [43]:
df.head()

,POSTED_BY,UNDER_CONSTRUCTION,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,2,BHK,1300.236407,1,1,12.969910,77.597960,55.0
1,Dealer,0,2,BHK,1275.000000,1,1,12.274538,76.644605,51.0
2,Owner,0,2,BHK,933.159722,1,1,12.778033,77.632191,43.0
3,Owner,0,2,BHK,929.921143,1,1,28.642300,77.344500,62.5
4,Dealer,1,2,BHK,999.009247,0,1,22.592200,88.484911,60.5


In [44]:
# df['numbers'] = df.index

In [45]:
df.head()

,POSTED_BY,UNDER_CONSTRUCTION,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,2,BHK,1300.236407,1,1,12.969910,77.597960,55.0
1,Dealer,0,2,BHK,1275.000000,1,1,12.274538,76.644605,51.0
2,Owner,0,2,BHK,933.159722,1,1,12.778033,77.632191,43.0
3,Owner,0,2,BHK,929.921143,1,1,28.642300,77.344500,62.5
4,Dealer,1,2,BHK,999.009247,0,1,22.592200,88.484911,60.5


In [46]:
# label_and_numbers_column = df[['TARGET(PRICE_IN_LACS)','numbers']]
# label_and_numbers_column

In [47]:
# without_labels = df.drop('TARGET(PRICE_IN_LACS)',axis=1)
# without_labels

In [48]:
# for c in without_labels.columns:
#     if without_labels[c].dtype=='object':
#         lbl = LabelEncoder()
#         lbl.fit(list(without_labels[c].values))
#         without_labels[c] = lbl.transform(without_labels[c].values)

In [49]:
# without_labels.head()

In [50]:
# new_df = pd.merge(without_labels, label_and_numbers_column, on="numbers")

In [51]:
# new_df[30000:40000]

In [52]:
for c in df.columns:
    if df[c].dtype=='object':
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)

In [53]:
all_corr = df.corr().abs()['TARGET(PRICE_IN_LACS)'].sort_values(ascending = False)

In [54]:
all_corr

TARGET(PRICE_IN_LACS)    1.000000
SQUARE_FT                0.409571
RESALE                   0.203275
BHK_NO.                  0.113654
POSTED_BY                0.093575
UNDER_CONSTRUCTION       0.054618
READY_TO_MOVE            0.054618
LONGITUDE                0.030505
LATITUDE                 0.017813
BHK_OR_RK                0.004453
Name: TARGET(PRICE_IN_LACS), dtype: float64

In [55]:
X = df.drop('TARGET(PRICE_IN_LACS)',axis=1)

In [56]:
y = df['TARGET(PRICE_IN_LACS)']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)
X_train.shape, X_test.shape

((23240, 9), (5810, 9))

In [58]:
X_train

,POSTED_BY,UNDER_CONSTRUCTION,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE
6830,1,0,2,0,1225.114855,1,1,18.097689,74.735239
14252,1,0,1,0,650.042626,1,1,19.218498,72.992114
25443,2,0,2,0,1250.000000,1,1,12.968000,80.150200
17749,1,0,1,0,689.655172,1,1,18.966670,72.833330
15019,1,0,3,0,1910.176236,1,1,30.662283,76.822397
...,...,...,...,...,...,...,...,...,...
10258,2,0,2,0,950.118765,1,1,27.180000,78.020000
16386,1,1,2,0,486.017646,0,0,24.690280,78.418890
17507,1,1,1,0,760.836617,0,1,21.170000,72.830000
29366,1,1,2,0,992.025640,0,1,19.120000,72.910000


In [59]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [60]:
class Linear_Regression:

    def __init__(self, learning_rate=0.1, iterations=100):
        self.learning_rate = learning_rate
        self.iterations = iterations

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.m, self.n = X.shape
        self.w = np.random.rand(self.n)
        self.b = 0

        for i in range(self.iterations):
            y_pred = np.dot(X, self.w) + self.b
            gW = - ( 2 * np.dot(X.T, (self.y - y_pred))) / self.m
            gb = - 2 * np.sum( self.y - y_pred ) / self.m
            self.w = self.w - self.learning_rate*gW
            self.b = self.b - self.learning_rate*gb

    def predict(self, X):
        return np.dot(X, self.w) + self.b

    def rmse(self, y, y_pred):
        return np.sqrt(np.mean(np.square(y - y_pred)))

    def r_squared(self, y, y_pred):
        return 1 - ((np.sum(np.square(y-y_pred)))/(np.sum(np.square(y-np.mean(y)))))

    def adjusted_r_squared(self, y, y_pred):
        return 1 - ((1-self.r_squared(y, y_pred))*(self.m - 1)/ (self.m - self.n - 1))

    @property
    def weights(self):
        return self.w

    @property
    def bias(self):
        return self.b

In [61]:
rf_model = RandomForestRegressor(random_state=42)
xgb_model = XGBRegressor(random_state=42)
custom_model = Linear_Regression()

In [62]:
rf_model.fit(X_test,y_test)

RandomForestRegressor(random_state=42)